# RNN
### compare with other NN methods, recurrent neuron network is more netural. 
#### In this notebook, I will to build up a RNN with the one hot encoded data

In [5]:
import numpy as np
import pandas as pd
from utils import *
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()
from sklearn.metrics import mean_squared_error
import torch                     # pytorch
import torch.nn as nn            # neural network components
import torch.optim as optim      # optimizers
import torch.nn.functional as F  # commonly used functions
from torch.utils.data import TensorDataset, DataLoader


### Get the one hot encoded data

In [3]:
df = pd.read_csv('use_data_dummy.csv').drop(columns = ['Unnamed: 0','store_nbr'])
train_df, test_df = make_cutoffs(df, 90, 15, stride = 30)

In [4]:
train_df.head(3)

,date,sales,onpromotion,oil_price,holiday,type_A,type_B,type_C,type_D,type_E,...,cluster_9,cluster_10,cluster_11,cluster_12,cluster_13,cluster_14,cluster_15,cluster_16,cluster_17,cutoff
0,2013-01-01,0.0,0,93.14,0.0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,2013-01-01
1,2013-01-01,0.0,0,93.14,0.0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,2013-01-01
2,2013-01-01,0.0,0,93.14,0.0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,2013-01-01


### build up the RNN

In [7]:
class RNN_Model(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, dropout_prob):
        super(RNNModel, self).__init__()

        # Defining the number of layers and the nodes in each layer
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim

        # RNN layers
        self.rnn = nn.RNN(
            input_dim, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob
        )
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initializing hidden state for first input with zeros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # Forward propagation by passing in the input and hidden state into the model
        out, h0 = self.rnn(x, h0.detach())

        # Reshaping the outputs in the shape of (batch_size, seq_length, hidden_size)
        # so that it can fit into the fully connected layer
        out = out[:, -1, :]

        # Convert the final state to our desired output shape (batch_size, output_dim)
        out = self.fc(out)
        return out
    
    
### 
class Optimization:
    def __init__(self, model, loss_fn, optimizer):
        self.model = model
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.train_losses = []
        self.val_losses = []
    
    def train_step(self, x, y):
        # Sets model to train mode
        self.model.train()

        # Makes predictions
        yhat = self.model(x)

        # Computes loss
        loss = self.loss_fn(y, yhat)

        # Computes gradients
        loss.backward()

        # Updates parameters and zeroes gradients
        self.optimizer.step()
        self.optimizer.zero_grad()

        # Returns the loss
        return loss.item()
    
    def train(self, train_loader, val_loader, batch_size=64, n_epochs=50, n_features=1):
        model_path = f'models/{self.model}_{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'

        for epoch in range(1, n_epochs + 1):
            batch_losses = []
            for x_batch, y_batch in train_loader:
                x_batch = x_batch.view([batch_size, -1, n_features]).to(device)
                y_batch = y_batch.to(device)
                loss = self.train_step(x_batch, y_batch)
                batch_losses.append(loss)
            training_loss = np.mean(batch_losses)
            self.train_losses.append(training_loss)

            with torch.no_grad():
                batch_val_losses = []
                for x_val, y_val in val_loader:
                    x_val = x_val.view([batch_size, -1, n_features]).to(device)
                    y_val = y_val.to(device)
                    self.model.eval()
                    yhat = self.model(x_val)
                    val_loss = self.loss_fn(y_val, yhat).item()
                    batch_val_losses.append(val_loss)
                validation_loss = np.mean(batch_val_losses)
                self.val_losses.append(validation_loss)

            if (epoch <= 10) | (epoch % 50 == 0):
                print(
                    f"[{epoch}/{n_epochs}] Training loss: {training_loss:.4f}\t Validation loss: {validation_loss:.4f}"
                )

        torch.save(self.model.state_dict(), model_path)